In [15]:
import os

In [16]:
%pwd

'd:\\software\\python_vs\\Bone_marrow_survival_prediction'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\software\\python_vs\\Bone_marrow_survival_prediction'

# entity

In [54]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path
    
    

In [55]:

from Bone_marrow_survival_prediction.constants import *
from Bone_marrow_survival_prediction.utils.comman import read_yaml, create_directories



In [56]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH, 
        schema_filepath = SCHEMA_FILE_PATH ):
        
       
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config['artifacts_root']])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']
        
        create_directories([config['root_dir']])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config['root_dir'],
            source_URL=config['source_URL'],
            local_data_file=config['local_data_file'],
            unzip_dir=config['unzip_dir'],
            
        )
        
        return data_ingestion_config


# components

In [57]:
import zipfile
import pandas as pd
import arff
import shutil
import os
import urllib.request as request
from pathlib import Path
from Bone_marrow_survival_prediction import logger
from Bone_marrow_survival_prediction.utils.comman import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    """def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            #logger.info(f"{filename} downloaded with following info: \n {headers}")
            logger.info(f"{filename} downloaded")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"w") as zip_ref:
            zip_ref.extractall(unzip_path)"""
            
    def download_file(self):
        # Create the local data file directory if it doesn't exist
        local_data_file_dir = os.path.dirname(self.config.local_data_file)
        if not os.path.exists(local_data_file_dir):
            os.makedirs(local_data_file_dir, exist_ok=True)

        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            print(f"{filename} downloaded! with the following info: \n{headers}")
        else:
            print(f"File already exists of size: {os.path.getsize(self.config.local_data_file)}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        print(f"Extracted all files to {unzip_path}")
            

# pipeline

In [58]:
try:
    #config_filepath = Path("D:\\software\\python_vs\\Bone_marrow_survival_prediction\\config\\config.yaml")
    #params_filepath = Path("D:\\software\\python_vs\\Bone_marrow_survival_prediction\\params.yaml")
    #schema_filepath = Path("D:\\software\\python_vs\\Bone_marrow_survival_prediction\\schema.yaml")
    
    #config = ConfigurationManager(config_filepath, params_filepath, schema_filepath)
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[ 2025-01-16 19:55:47,312: INFO: comman: yaml file config\config.yaml read_successfully]
[ 2025-01-16 19:55:47,312: INFO: comman: yaml file params.yaml read_successfully]
[ 2025-01-16 19:55:47,312: INFO: comman: yaml file schema.yaml read_successfully]
File already exists of size: 225855


BadZipFile: File is not a zip file

In [11]:
import os
import urllib.request as request
import zipfile
import yaml
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path
    

def read_yaml(filepath):
    with open(filepath, 'r') as file:
        return yaml.safe_load(file)

def create_directories(directories):
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

class DataIngestionConfig:
    def __init__(self, root_dir, source_URL, local_data_file, unzip_dir):
        self.root_dir = root_dir
        self.source_URL = source_URL
        self.local_data_file = local_data_file
        self.unzip_dir = unzip_dir

class ConfigurationManager:
    def __init__(self, config_filepath, params_filepath, schema_filepath):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config['artifacts_root']])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']
        
        create_directories([config['root_dir']])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config['root_dir'],
            source_URL=config['source_URL'],
            local_data_file=config['local_data_file'],
            unzip_dir=config['unzip_dir']
        )
        
        return data_ingestion_config

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        # Create the local data file directory if it doesn't exist
        local_data_file_dir = os.path.dirname(self.config.local_data_file)
        if not os.path.exists(local_data_file_dir):
            os.makedirs(local_data_file_dir, exist_ok=True)

        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            print(f"{filename} downloaded! with the following info: \n{headers}")
        else:
            print(f"File already exists of size: {os.path.getsize(self.config.local_data_file)}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        print(f"Extracted all files to {unzip_path}")
        
        
try:
    config_filepath = Path("D:\\software\\python_vs\\Bone_marrow_survival_prediction\\config\\config.yaml")
    params_filepath = Path("D:\\software\\python_vs\\Bone_marrow_survival_prediction\\params.yaml")
    schema_filepath = Path("D:\\software\\python_vs\\Bone_marrow_survival_prediction\\schema.yaml")
    
    config = ConfigurationManager(config_filepath, params_filepath, schema_filepath)
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

# Example usage


artifacts/data_ingestion/data.zip downloaded! with the following info: 
Server: GitHub.com
Date: Thu, 16 Jan 2025 01:48:55 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"8e54a348b66f7f0458f5a2001992fd79"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: no-referrer-when-downgrade
Content-Security-Policy: default-src 'none'; base-uri 'self'; child-src github.com/assets-cdn/worker/ github.com/webpack/ github.com/assets/ gist.github.com/assets-cdn/worker/; connect-src 'self' uploads.github.com www.githubstatus.com collector.github.com raw.githubusercontent.com api.github.com github-cloud.s3.amazonaws.com github-production-repository-file-5c1aeb.s3.amazonaws.com github-production-upload-manifest-file-7fdce7.s3.amazonaws.com g

BadZipFile: File is not a zip file